<a href="https://colab.research.google.com/github/onuralpArsln/finTech/blob/main/tryOtherMethods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Considering you already have your files inside github links

In [ ]:
## Güncel data linkleri <br> <br>

file1 = "https://raw.githubusercontent.com/onuralpArsln/finTech/refs/heads/main/data/lastWeek.csv?token=GHSAT0AAAAAACLQZ7SITTNWYGU4AFMRQCZ2ZXPXZZA"
file2 = "https://raw.githubusercontent.com/onuralpArsln/finTech/refs/heads/main/data/twoWeek.csv?token=GHSAT0AAAAAACLQZ7SIAZ6M2GER76YGBQ4EZXPXZ3A"
file3 = "https://raw.githubusercontent.com/onuralpArsln/finTech/refs/heads/main/data/fourWeek.csv?token=GHSAT0AAAAAACLQZ7SIZUNDS5LREKVI4BA4ZXPXY2Q"

allLinks =[file1, file2, file3]

Get data into arrays

In [ ]:
import pandas as pd


series_list=[]
for url in allLinks:
  df= pd.read_csv(url, header=None)
  data=df.values.tolist()
  single_dimensional_list = [item[0] for item in data]
  series_list.append( single_dimensional_list)



In [ ]:
test_dfs=[]
train_dfs=[]

for series in series_list:
 
    last_10 = series[-10:]
  
    rest = series[:-10]
    # append to dataset
    test_dfs.append(last_10)
    train_dfs.append(rest)

print(test_dfs[0])

Buradan aşağısı chat gpt copy8 pastesi ondan düzeltme lazım 



1. LSTM (Long Short-Term Memory) Networks

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

# Convert train_dfs to a DataFrame
prices = pd.DataFrame(train_dfs, columns=['Close'])

# Scale data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_prices = scaler.fit_transform(prices)

# Create sequences
def create_dataset(data, time_step=1):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        a = data[i:(i + time_step), 0]
        X.append(a)
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

X, y = create_dataset(scaled_prices, time_step=10)
X = X.reshape(X.shape[0], X.shape[1], 1)

# Build LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X.shape[1], 1)))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Train model
model.fit(X, y, epochs=50, batch_size=32)

# Make predictions
predictions = model.predict(X)
predictions = scaler.inverse_transform(predictions)


2. GARCH (Generalized Autoregressive Conditional Heteroskedasticity)

In [ ]:
import pandas as pd
from arch import arch_model

# Convert train_dfs to a DataFrame
prices = pd.DataFrame(train_dfs, columns=['Close'])
returns = prices['Close'].pct_change().dropna()

# Fit GARCH model
model = arch_model(returns, vol='Garch', p=1, q=1)
model_fit = model.fit()

# Make volatility predictions
predicted_volatility = model_fit.conditional_volatility


3. XGBoost

In [ ]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Convert train_dfs to a DataFrame
prices = pd.DataFrame(train_dfs, columns=['Close'])
prices['Lag1'] = prices['Close'].shift(1)  # Create lagged feature
prices = prices.dropna()

# Prepare features and target
X = prices[['Lag1']]
y = prices['Close']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train XGBoost model
model = XGBRegressor()
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')
